In [1]:
import findspark
findspark.init()

import json
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_utc_timestamp, from_unixtime, to_date, sum,count, mean, countDistinct, round
from pyspark import SparkContext
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.types import IntegerType,StructField,StructType, LongType, FloatType
from pyspark.sql.functions import col, when

from pyspark.sql.functions import monotonically_increasing_id, coalesce, date_trunc, regexp_extract
import snap
from os import path
import glob

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
#from bitcoin_utils import bitcoin_utils



#import bitcoin_utils

In [2]:
spark= SparkSession \
    .builder \
    .config("spark.driver.memory", "100g") \
    .config("spark.executor.memory", "100g") \
    .config("spark.cores.max", "10") \
    .getOrCreate()

In [3]:
#M = bitcoin_create_transaction_file(spark,sc)

In [4]:
import bitcoin_create_transaction_file

In [5]:
M = bitcoin_create_transaction_file.bitcoin_create_transaction_file(spark)

In [11]:
#M.createAd2Ad("/media/ssd2/bitcoinRemy/ad2ad",year_min=2009,year_max=2009)
####M.createAd2Ad("/media/ssd2/bitcoinRemy/ad2ad")

In [5]:
with_identity = M.append_identity("/media/ssd2/bitcoinRemy/ad2ad",
                  "/media/ssd2/bitcoinRemy/to_end/ad2cl2identity/")
with_identity.write.mode("overwrite").save("/media/ssd2/bitcoinRemy/tr_with_identity")

In [7]:
price_file = spark.read.csv("/media/ssd2/bitcoinRemy/btc_prices.csv",sep=",",header=True)\
    .select(["date","PriceUSD"])\
    .withColumn("PriceUSD", col("PriceUSD").cast(FloatType()))

before = spark.read.load("/media/ssd2/bitcoinRemy/tr_with_identity")#.select(["time","value"])
result = M.append_USD_value(before,price_file,"PriceUSD")
               
result.write.mode("overwrite").save("/media/ssd2/bitcoinRemy/tr_with_prices")

In [9]:
!rm -R /media/ssd2/bitcoinRemy/tr_with_identity

In [9]:
dataset = spark.read.load("/media/ed5/remyBitcoin/whole_dataset/")#.filter(col("year")<2010)
to_write = M.create_spending_details(dataset)
to_write.write.save("/media/ssd2/bitcoinRemy/spending_details")

In [6]:
#spark.read.load("/media/ssd2/bitcoinRemy/tr_with_prices").select(["hash","value","date","output_id"]).filter(year("date")<2010).write.mode("overwrite").save("/media/ssd2/bitcoinRemy/test")

pouet = M.append_spend_details(spark.read.load("/media/ssd2/bitcoinRemy/tr_with_prices"),
                      spark.read.load("/media/ssd2/bitcoinRemy/spending_details"))
pouet.write.save("/media/ssd2/bitcoinRemy/tr_with_spending_details")

In [8]:
!rm -R /media/ssd2/bitcoinRemy/tr_with_prices

In [6]:
wecat=spark.read.csv("/media/ssd2/bitcoinRemy/walletExplorer_category.csv",sep=";")\
    .withColumnRenamed("_c0","name").withColumnRenamed("_c1","cat")

to_write = M.append_categories(spark.read.load("/media/ssd2/bitcoinRemy/tr_with_spending_details"),wecat)
                    
to_write.write.save("/media/ssd2/bitcoinRemy/tr_with_everything")    

In [13]:
!rm -R /media/ssd2/bitcoinRemy/tr_with_spending_details

+--------------------+---------+
|                name|      cat|
+--------------------+---------+
|           Huobi.com|Exchanges|
|         Bittrex.com|Exchanges|
|        Poloniex.com|Exchanges|
|            Luno.com|Exchanges|
|           BTC-e.com|Exchanges|
|          Kraken.com|Exchanges|
|   LocalBitcoins.com|Exchanges|
|        Bitstamp.net|Exchanges|
|MercadoBitcoin.co...|Exchanges|
|        BitZlato.com|Exchanges|
|         Cryptsy.com|Exchanges|
|          Bitcoin.de|Exchanges|
|              Cex.io|Exchanges|
|         Binance.com|Exchanges|
|        BtcTrade.com|Exchanges|
|           YoBit.net|Exchanges|
|          OKCoin.com|Exchanges|
|            BTCC.com|Exchanges|
|            BX.in.th|Exchanges|
|          HitBtc.com|Exchanges|
+--------------------+---------+
only showing top 20 rows



In [6]:
M.write_temporally(spark.read.load("/media/ssd2/bitcoinRemy/tr_with_everything/"),"/media/ssd2/bitcoinRemy/tr_everything_by_day")